# Pull Performance Report Results with GS Quant

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import pandas as pd
from IPython.display import display
import warnings

from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.session import GsSession, Environment


client = None
secret = None


## External users must fill in their client ID and secret below and comment out the line below
# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(
    Environment.PROD,
    client_id=client,
    client_secret=secret,
)
warnings.filterwarnings("ignore", category=RuntimeWarning)


print('GS Session initialized.')

In [2]:
pm = PortfolioManager('ENTER PORTFOLIO ID')
performance_report = pm.get_performance_report()
start_date = performance_report.earliest_start_date
end_date = performance_report.latest_end_date

### Step 2: Pull daily exposures and PnL data: 

Now that we have our performance report, we can leverage the unique functionalities of the PerformanceReport class to pull daily exposure and PnL data:

In [ ]:
all_exposures = performance_report.get_many_measures(
    start_date=start_date, end_date=end_date, measures=['pnl', 'grossExposure', 'netExposure']
)

print(all_exposures)
all_exposures.plot(title='Performance Breakdown')

### Step 3: Pull Brinson Attribution Data 
Now let's pull Brinson Attribution data to analyze the PnL of your portfolio compared to a benchmark, which can be any equity ETF, Index, or Basket in Marquee:

*Important Note: An error in the code below might occur if time period for which you are querying is too long*

In [ ]:
asset = Asset.get(id_value='MXWO', id_type=AssetIdentifier.TICKER)

brinson_attribution_results = performance_report.get_brinson_attribution(
    benchmark=asset.get_marquee_id(), include_interaction=True, start_date=start_date, end_date=end_date
)

display(pd.DataFrame(brinson_attribution_results))

### Step 4: Pull Asset Level Performance Data

We can pull asset level data from the performance report.

You can extract asset level data of quantity, netWeight, netExposure, etc.., using `get_positions_data`.


In [ ]:
asset_level_data = performance_report.get_positions_data(
    start=performance_report.earliest_start_date, end=performance_report.earliest_start_date
)

asset_level_data

You can also pull asset level contribution to PnL using the `pnl_contribution` function used below.

In [ ]:
pnl_contribution = performance_report.get_pnl_contribution(
    start_date=performance_report.earliest_start_date, end_date=performance_report.latest_end_date
)

pnl_contribution